#0. 라이브러리 임포트

In [4]:
import warnings

In [5]:
import pandas as pd
import numpy as np
from scipy import stats

#1. one-sample t-test



*   정의
  * 기정 사실의 평균과 주어진 표본의 평균 비교
*   문제
  * 어느 핸드폰 케이스 제조사가 휴대폰 케이스의 무게가 15g이라고 광고 but 한 고객이 정교한 저울로 휴대폰 케이스의 중량을 재보니 15g을 초과하는 것을 발견

  * 핸드폰 케이스의 평균 무게가 진짜 15g인지 중량의 평균검정 실행


##1. 가설 설정
* H0 : 휴대폰케이스의 중량은 15g이다
* H1 : 휴대폰케이스의 중량은 15g과 차이가 있다.

## 2. 데이터 로드

In [ ]:
src_path = 'https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/python_da/mobile_weight.csv'
df = pd.read_csv(src_path, sep=',')

In [ ]:
df

,weight
0,14.350632
1,14.323534
2,14.955139
3,14.577219
4,15.667932
...,...
129,15.175354
130,15.576407
131,14.483923
132,14.161478


##3. t-test

In [ ]:
# 중량의 평균 -> 표본 평균
print(df['weight'].mean())

15.03196778256587


In [ ]:
E_H0 = 15 # 귀무가설 기댓값

# spicy 패키지의 stats 서브 모듈 (표본평균, 기존의 평균)
stats.ttest_1samp(df['weight'], E_H0)

TtestResult(statistic=0.6461573171531722, pvalue=0.5192908627711725, df=133)

만약 유의수준이 0.05라면 pvalue > 유의수준이 되므로 H0이 기각이 되지 않으므로 H0이 채택되고 새로운 가설 H1은 기각된다.

#2. two-sample t-test



*   정의
  * 두 개의 독립적인 표본 사이에 **하나의 종속변수의 평균**이 동일 한가를 검정
*   문제
  * A 대리점 매출액에 대해 프로모션 수행시와 미수행 시를 비교해 **매출액(종속변수)**이 통계적으로 유의하게 커지는, 차이가 나는지 검정


##1. 가설설정




*   프모 o x 평균 매출액 차이 無
=> mean(Y)-mean(N) = 0
*   프모 o x 평균 매출액 차이 有
=> mean(Y)-mean(N) != 0
*  프모 o > 프모 x
=> mean(Y) - mean(N) > 0



##2. 데이터 로드

In [7]:
src_path = src_path = 'https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/python_da/shop_ab_sales.csv'
df = pd.read_csv(src_path, sep=',')

In [9]:
df

,pos_date,pos_hour,date_hour,shop_nm,sales_amt,prom_yn
0,20170601,10,2017060110,B,330.0,N
1,20170602,13,2017060213,A,992.0,N
2,20170602,22,2017060222,A,1213.0,N
3,20170602,22,2017060222,B,2095.0,N
4,20170602,13,2017060213,B,2426.0,N
...,...,...,...,...,...,...
784,20170612,15,2017061215,A,1103.0,N
785,20170614,11,2017061411,B,330.0,N
786,20170614,11,2017061411,A,110.0,N
787,20170620,12,2017062012,B,661.0,N


In [23]:
sample_df = df.loc[df['shop_nm']=='A']
print(sample_df['prom_yn'].value_counts())

prom_yn
N    317
Y     79
Name: count, dtype: int64


In [25]:
prom_Y = sample_df.loc[df['prom_yn']=='Y']
prom_N = sample_df.loc[df['prom_yn']=='N']

##3. 등분산 검정

등분산 검정함수 param type : ndarray

In [26]:
np_arr_prom_Y = np.array(prom_Y['sales_amt'])
np_arr_prom_N = np.array(prom_N['sales_amt'])

In [27]:
print(stats.bartlett(np_arr_prom_Y, np_arr_prom_N))

BartlettResult(statistic=0.3505843659785136, pvalue=0.5537825209464317)


p-value가 유의수준인 0.05보다 크므로 H0이 채택된다. 즉 등분산이다

##4. t-test

In [34]:
# equal_var = True : 등분산 o
print(stats.ttest_ind(np_arr_prom_N, np_arr_prom_Y,equal_var=True))

TtestResult(statistic=-2.0298638294827196, pvalue=0.04304185346100201, df=394.0)


p-value < 유의수준 -> H1 채택 = 프로모 ox의 판매액 평균 가격 차이 有

cf ) df (degrees of freedom)

#3. paired sample t-test



*   정의
  * 실험 전 집단과 실험 후 집단의 쌍체로 이뤄진 두 표본 집단의 평균 비교
*   문제
  * A대리점은 제품의 판매량을 높이기 위해 일정기간 프로모션을 수행

  * B대리점과 비교하여 매출액이 증가하였는지 비교


##1. 가설설정
* H0 : 프로모션 진행시, A대리점과 B대리점의 평균 매출액 차이는 없다.

* H1 : 프로모션 진행시, A대리점과 B 대리점의 평균 매출액 차이는 있다
 or 프로모션 진행시, A대리점의 평균매출액이 더 높다.

##2. 데이터 로드

In [35]:
df.head(3)

,pos_date,pos_hour,date_hour,shop_nm,sales_amt,prom_yn
0,20170601,10,2017060110,B,330.0,N
1,20170602,13,2017060213,A,992.0,N
2,20170602,22,2017060222,A,1213.0,N


A와 B 대리점 분리

In [36]:
a_df = df.loc[df['shop_nm']=='A']
b_df = df.loc[df['shop_nm']=='B']

In [40]:
#대응표본 개수 확인
print(len(a_df), len(b_df))

396 393


In [41]:
#대응표본 만들기
merge_df = pd.merge(a_df, b_df, on='date_hour')

In [42]:
merge_df

,pos_date_x,pos_hour_x,date_hour,shop_nm_x,sales_amt_x,prom_yn_x,pos_date_y,pos_hour_y,shop_nm_y,sales_amt_y,prom_yn_y
0,20170602,13,2017060213,A,992.0,N,20170602,13,B,2426.0,N
1,20170602,22,2017060222,A,1213.0,N,20170602,22,B,2095.0,N
2,20170603,16,2017060316,A,3970.0,N,20170603,16,B,4742.0,N
3,20170604,19,2017060419,A,3529.0,N,20170604,19,B,2426.0,N
4,20170605,12,2017060512,A,1433.0,N,20170605,12,B,1764.0,N
...,...,...,...,...,...,...,...,...,...,...,...
386,20170630,15,2017063015,A,2867.0,Y,20170630,15,B,2426.0,Y
387,20170605,11,2017060511,A,110.0,N,20170605,11,B,661.0,N
388,20170611,12,2017061112,A,1323.0,N,20170611,12,B,1213.0,N
389,20170612,15,2017061215,A,1103.0,N,20170612,15,B,2426.0,N


##3 t-test

In [44]:
# A대리점의 프로모션 진행기간의 데이터 추출
# A가 프로모션했을 때만 고려하는 거라서 B의 조건을 안넣었구
sample1_df = merge_df.loc[merge_df['prom_yn_x']=='Y']

In [54]:
sample2_df=merge_df.loc[(merge_df['prom_yn_x']=='Y')& (merge_df['prom_yn_y']=='Y')]

In [45]:
print(stats.ttest_rel(sample1_df['sales_amt_x'], sample1_df['sales_amt_y']))

TtestResult(statistic=5.58245928204439, pvalue=3.505172868522075e-07, df=76)


프로모 효과 있음 p-value < 유의 수준

단측검정 실행(mean(A))>mean(B))

statistic = sample1 - sample2(=mean(A)-mean(B))

함수의 p-value*1/2 = 단측검정의 결과

** mean(A)-mean(B) > 0 이 맞다 **

# 추가
* two-sample t-test는 두 집단간의 평균이 0인가를 확인하는 검증이고 paired-sample t-test는 두 집단의 평균의 차이가 0이 되는지를 확인하는 것이다.

* 날짜별로 차이가 있는지 없는 지 확인하거나 같은 인덱싱에 있는 데이터들의 차이를 확인하고 싶을 때 paired-sample test를 사용하는 것이 좋다. two-sample t-test는 뭉탱이로 집단간의 차이를 확인하는거니깐,,,


-------------
* t-test default 는 양측검정이다.